# 2. Tensorflow Prophet Testing

## I - Setup

In [1]:
# Import of librairies
import tensorflow as tf
import mysql.connector as mariadb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime
import math
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tqdm import tqdm
from joblib import dump

class sql_query:
    def __init__(self, credentials_path):
        self.db_credentials = pd.read_csv(credentials_path, index_col="Field")
      
    
    def __call__(self, query):
        
        mariadb_connection = mariadb.connect(
            user=self.db_credentials.loc["user"][0],
            password=self.db_credentials.loc["password"][0],
            host=self.db_credentials.loc["host"][0],
            port=3306,
            db = "db_velib")
        
        self.cursor = mariadb_connection.cursor()
    
        cursor = self.cursor
        cursor.execute(query)
        field_names = [i[0] for i in cursor.description]
        df = pd.DataFrame(cursor, columns=field_names)
        return df
    
# Transforming the input data in the proper format 



def data_preparation(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i, step)
        data.append(dataset[indices])

        if single_step:
            labels.append(target[i+target_size])
        else:
            labels.append(target[i:i+target_size])

    return np.array(data), np.array(labels)


def measure_rmse(actual, predicted):
    return math.sqrt(mean_squared_error(actual, predicted))

def model_training(station_id, day_of_testing, past_history, future_target):


    tf.random.set_seed(13)
    past_history = 36
    future_target = 6
    STEP = 1
    BATCH_SIZE = 32
    BUFFER_SIZE = 100000
    EPOCHS = 6
    EVALUATION_INTERVAL = 200

    request = sql_query("../../aws_mariadb_crendentials.csv")

    query = """
    SELECT DISTINCT date_of_update, nb_total_free_bikes FROM velib_realtime
    WHERE station_id = {}
    AND date_of_update > DATE('2020-05-05')
    AND date_of_update <= DATE_ADD(DATE('{}'), INTERVAL 1 DAY)
    AND MINUTE(date_of_update)%5=0
    ORDER BY date_of_update ASC
    """.format(station_id, day_of_testing)

    df = request(query)
    df.index = df['date_of_update']
    df = df.nb_total_free_bikes
    
    
    
    TRAIN_SPLIT = round(df.shape[0]*0.7)

    # StandardScaler transformation of the dataset

    std = StandardScaler()
    std.fit(df[:TRAIN_SPLIT].values.reshape(-1,1))
    df = std.transform(df.values.reshape(-1,1))

    # Creating proper format data

    x_train, y_train = data_preparation(df, df[1:], 0, TRAIN_SPLIT,
                                               past_history,
                                               future_target, STEP)
    x_val, y_val = data_preparation(df, df[1:], TRAIN_SPLIT, None,
                                           past_history,
                                           future_target, STEP)

    # Creating format for NN intput

    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
    x_val = x_val.reshape(x_val.shape[0], x_val.shape[1], 1)

    # Creating batches for tensorflow use

    train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_data = train_data.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

    val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))
    val_data = val_data.batch(BATCH_SIZE).repeat()

    # Modeling A
    
    LSTM_model_A = tf.keras.models.Sequential([
        tf.keras.layers.LSTM(32, input_shape=x_train.shape[-2:]),
        tf.keras.layers.Dense(future_target)
    ])

    LSTM_model_A.compile(optimizer='adam', loss='mean_squared_error')

    LSTM_model_A_history = LSTM_model_A.fit(train_data, epochs=EPOCHS,
                                                steps_per_epoch=EVALUATION_INTERVAL,
                                                validation_data=val_data,
                                                validation_steps=200)
    
    # Modeling B

    LSTM_model_B = keras.Sequential()
    LSTM_model_B.add(
      keras.layers.Bidirectional(
        keras.layers.LSTM(
          units=64,
          input_shape=(x_train.shape[-2:])
        )
      )
    )
    LSTM_model_B.add(keras.layers.Dropout(rate=0.2))
    LSTM_model_B.add(keras.layers.Dense(units=future_target))
    LSTM_model_B.compile(loss='mean_squared_error', optimizer='adam')

    LSTM_model_B_history = LSTM_model_B.fit(train_data, epochs=EPOCHS,
                                            steps_per_epoch=EVALUATION_INTERVAL,
                                            validation_data=val_data,
                                            validation_steps=200)
    
    
    return LSTM_model_A, LSTM_model_A_history, LSTM_model_B, LSTM_model_B_history, std

def results_filling(df_results):
    
    for i in df_results.index:
        
        try:
            # interval // Taking the last 180 values
            past_for_prediction = df[(i - datetime.timedelta(minutes=past_history*5+60)):i][-past_history:].values
            past_for_prediction_encoded = std.transform(past_for_prediction.reshape(-1, 1))

            # Prediction of A
            results_A = LSTM_model_A.predict(past_for_prediction_encoded.reshape(1,past_history,1))[0]
            results_A = std.inverse_transform(results_A)

            # Prediction of B
            results_B = LSTM_model_B.predict(past_for_prediction_encoded.reshape(1,past_history,1))[0]
            results_B = std.inverse_transform(results_B)

            df_results.prediction_A[i] = results_A
            df_results.prediction_B[i] = results_B
            df_results.real_values[i] = df[i: i + datetime.timedelta(minutes=60)][0:future_target].values

            df_results.loc[i].metrics_A = measure_rmse(df_results.loc[i].real_values, df_results.loc[i].prediction_A)
            df_results.loc[i].metrics_B = measure_rmse(df_results.loc[i].real_values, df_results.loc[i].prediction_B)
        except:
            print('error at', i)
            
            df_results.loc[i].metrics_A = None
            df_results.loc[i].metrics_B = None
        
    return df_results


In [2]:
# Extracting the list of the stations

request = sql_query("../../aws_mariadb_crendentials.csv")
query = """
SELECT DISTINCT station_id FROM velib_realtime
"""
df= request(query)
# Removing bad values
df= df.drop(0)
df = df.drop(1391)
list_of_stations = list(df.station_id)
print(list_of_stations[0:5])

[1001, 1002, 1003, 1006, 1007]


In [ ]:
# Main pipelinhe

# Variables
day_of_testing = '2020-05-17'
past_history = 36
future_target = 6

# Request for database

for station_id in tqdm(list_of_stations):
    
    request = sql_query("../../aws_mariadb_crendentials.csv")

    query = """
    SELECT DISTINCT date_of_update, nb_total_free_bikes FROM velib_realtime
    WHERE station_id = {}
    AND date_of_update > DATE('2020-05-05')
    AND date_of_update <= DATE_ADD(DATE('{}'), INTERVAL 1 DAY)
    ORDER BY date_of_update ASC
    """.format(station_id, day_of_testing)

    df = request(query)
    df.index = df['date_of_update']
    df = df.nb_total_free_bikes
    


    df_results = pd.DataFrame(columns=['prediction_A', 'prediction_B', 'real_values', 'metrics_A', 'metrics_B'], index=pd.date_range(day_of_testing+' 06:00:00', periods=64, freq='15Min'))

    # Training

    LSTM_model_A, LSTM_model_A_history, LSTM_model_B, LSTM_model_B_history, std = model_training(station_id, day_of_testing, past_history, future_target)

    # Exporting results
    
    LSTM_model_A.save('/home/exalis/Github/velib-prediction-v2/4. Models/Tensorflow Univariate - {} - {} - LSTM_A.h5'.format(day_of_testing, station_id))
    LSTM_model_B.save('/home/exalis/Github/velib-prediction-v2/4. Models/Tensorflow Univariate - {} - {} - LSTM_B.h5'.format(day_of_testing, station_id))
    dump(std, '/home/exalis/Github/velib-prediction-v2/4. Models/Tensorflow Univariate - {} - {} - std.joblib'.format(day_of_testing, station_id)) 
    
   # df_results.to_csv("/home/exalis/Github/velib-prediction-v2/4. Models/Tensorflow Univariate Results - {} - {}.csv".format(day_of_testing, station_id))
    
    
    print('finished ', station_id)

  0%|          | 0/1390 [00:00<?, ?it/s]

Train for 200 steps, validate for 200 steps
Epoch 1/6
200/200 [==============================] - 3s 16ms/step - loss: 0.2695 - val_loss: 0.4160
Epoch 2/6
200/200 [==============================] - 1s 7ms/step - loss: 0.1143 - val_loss: 0.3986
Epoch 3/6
200/200 [==============================] - 1s 7ms/step - loss: 0.1042 - val_loss: 0.3974
Epoch 4/6
200/200 [==============================] - 1s 7ms/step - loss: 0.0991 - val_loss: 0.4049
Epoch 5/6
200/200 [==============================] - 1s 7ms/step - loss: 0.0938 - val_loss: 0.4122
Epoch 6/6
200/200 [==============================] - 1s 7ms/step - loss: 0.1011 - val_loss: 0.3772
Train for 200 steps, validate for 200 steps
Epoch 1/6
200/200 [==============================] - 4s 21ms/step - loss: 0.2452 - val_loss: 0.4162
Epoch 2/6
200/200 [==============================] - 2s 10ms/step - loss: 0.1272 - val_loss: 0.4086
Epoch 3/6
200/200 [==============================] - 2s 10ms/step - loss: 0.1114 - val_loss: 0.3928
Epoch 4/6
200/200

  0%|          | 1/1390 [00:25<9:50:25, 25.50s/it]

finished  1001
Train for 200 steps, validate for 200 steps
Epoch 1/6
200/200 [==============================] - 3s 13ms/step - loss: 0.2516 - val_loss: 0.2853
Epoch 2/6
200/200 [==============================] - 1s 7ms/step - loss: 0.1271 - val_loss: 0.2450
Epoch 3/6
200/200 [==============================] - 1s 7ms/step - loss: 0.1133 - val_loss: 0.2336
Epoch 4/6
200/200 [==============================] - 1s 7ms/step - loss: 0.1094 - val_loss: 0.2427
Epoch 5/6
200/200 [==============================] - 1s 7ms/step - loss: 0.1078 - val_loss: 0.2244
Epoch 6/6
200/200 [==============================] - 1s 7ms/step - loss: 0.0990 - val_loss: 0.2106
Train for 200 steps, validate for 200 steps
Epoch 1/6
200/200 [==============================] - 4s 20ms/step - loss: 0.2449 - val_loss: 0.2446
Epoch 2/6
200/200 [==============================] - 2s 10ms/step - loss: 0.1382 - val_loss: 0.2654
Epoch 3/6
200/200 [==============================] - 2s 10ms/step - loss: 0.1270 - val_loss: 0.2280
Ep

  0%|          | 2/1390 [00:50<9:44:18, 25.26s/it]

...

finished  17017
Train for 200 steps, validate for 200 steps
Epoch 1/6
200/200 [==============================] - 3s 13ms/step - loss: 0.2281 - val_loss: 0.1419
Epoch 2/6
200/200 [==============================] - 1s 7ms/step - loss: 0.0989 - val_loss: 0.1507
Epoch 3/6
200/200 [==============================] - 1s 7ms/step - loss: 0.0861 - val_loss: 0.1126
Epoch 4/6
200/200 [==============================] - 1s 7ms/step - loss: 0.0825 - val_loss: 0.1176
Epoch 5/6
200/200 [==============================] - 1s 7ms/step - loss: 0.0759 - val_loss: 0.1161
Epoch 6/6
200/200 [==============================] - 1s 7ms/step - loss: 0.0755 - val_loss: 0.1210
Train for 200 steps, validate for 200 steps
Epoch 1/6
200/200 [==============================] - 4s 21ms/step - loss: 0.2169 - val_loss: 0.1416
Epoch 2/6
200/200 [==============================] - 2s 10ms/step - loss: 0.1060 - val_loss: 0.1380
Epoch 3/6
200/200 [==============================] - 2s 10ms/step - loss: 0.0961 - val_loss: 0.1220
E

 55%|█████▌    | 768/1390 [5:19:15<4:31:29, 26.19s/it]

finished  17018
Train for 200 steps, validate for 200 steps
Epoch 1/6
200/200 [==============================] - 3s 13ms/step - loss: 0.2174 - val_loss: 0.1428
Epoch 2/6
200/200 [==============================] - 1s 7ms/step - loss: 0.0864 - val_loss: 0.1365
Epoch 3/6
200/200 [==============================] - 1s 7ms/step - loss: 0.0762 - val_loss: 0.1393
Epoch 4/6
200/200 [==============================] - 1s 7ms/step - loss: 0.0716 - val_loss: 0.1293
Epoch 5/6
200/200 [==============================] - 1s 7ms/step - loss: 0.0696 - val_loss: 0.1313
Epoch 6/6
200/200 [==============================] - 1s 7ms/step - loss: 0.0659 - val_loss: 0.1322
Train for 200 steps, validate for 200 steps
Epoch 1/6
200/200 [==============================] - 4s 20ms/step - loss: 0.1931 - val_loss: 0.1534
Epoch 2/6
200/200 [==============================] - 2s 10ms/step - loss: 0.0955 - val_loss: 0.1504
Epoch 3/6
200/200 [==============================] - 2s 10ms/step - loss: 0.0879 - val_loss: 0.1503
E

 55%|█████▌    | 769/1390 [5:19:40<4:28:40, 25.96s/it]

finished  17019
